# <font color='red'>Import Needed Libraries</font>

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OrdinalEncoder


import warnings
warnings.filterwarnings('ignore')

 # <font color='red'>Upload and Clean Data</font>

In [2]:
main_df = pd.read_excel("Assignment.xlsx")
main_df.head()

,Date,improvement,The trainer communicated clearly and was easy to understand,The trainer presented in away that held my interest,The pace of this session was appropriate,Examples provided were practical and realistic,The session included ample opportunities for active involvement,There was ample time for questions,The course objectives were met,It provided valuable information that can be transferred to my job\n,It was logically sequenced,Your ability to apply the acquired knowledge or training received,How do you rate the e-learning experience overall?,Any other comments to appreciator help us improve the training service provided:,sentiment
0,27-04-2023,64.0,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,I learned a lot on this training,Positive
1,27-04-2023,64.0,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,Excellent,none,Neutral
2,27-04-2023,64.0,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,EXCELLENT COURSE,Positive
3,27-04-2023,64.0,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,NaN,Neutral
4,27-04-2023,64.0,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Good Session,Positive


In [3]:
main_df = main_df.drop(['Date', 'improvement'], axis =1 )

In [4]:
new_column_names = {'The trainer communicated clearly and was easy to understand': 'communication',
                    'The trainer presented in away that held my interest': 'presentation',
                    'The pace of this session was appropriate': 'pace',
                    'Examples provided were practical and realistic': 'examples',
                    'The session included ample opportunities for active involvement': 'involvemen',
                    'The course objectives were met': 'objective',
                    'It provided valuable information that can be transferred to my job\n': 'valuable information',
                    'It was logically sequenced': 'sequence',
                    'Your ability to apply the acquired knowledge or training received': 'apply',
                    'How do you rate the e-learning experience overall?': 'e-learning',
                    'Any other comments to appreciator help us improve the training service provided:': 'comment',
                    'sentiment': 'target',
                    'There was ample time for questions': 'questions'
                   }
main_df.rename(columns=new_column_names, inplace=True)

In [5]:
main_df.tail()

,communication,presentation,pace,examples,involvemen,questions,objective,valuable information,sequence,apply,e-learning,comment,target
2389,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,It was Great opportunity to understand more ab...,NaN
2390,Good,Excellent,Good,Good,Good,Excellent,Excellent,Good,Good,Excellent,Excellent,NaN,NaN
2391,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,NaN,NaN
2392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2393,2,4,4,6,7,6,5,9,5,5,7,NaN,NaN


In [6]:
len(main_df)

2394

In [7]:
main_df['target'].value_counts()

Positive    1095
Neutral      422
Negative      60
Name: target, dtype: int64

## Check target column, and change "No" comment to be with "Natural" target

In [8]:
len(main_df[main_df['comment'].isin(['no', 'No', 'NO'])])

85

In [9]:
main_df.loc[main_df["comment"].isin(['no', 'No', 'NO']), "target"] = "Neutral"

In [10]:
main_df.head()

,communication,presentation,pace,examples,involvemen,questions,objective,valuable information,sequence,apply,e-learning,comment,target
0,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,I learned a lot on this training,Positive
1,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,Excellent,none,Neutral
2,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,EXCELLENT COURSE,Positive
3,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,NaN,Neutral
4,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Good Session,Positive


In [11]:
main_df['target'].value_counts()

Positive    1095
Neutral      463
Negative      29
Name: target, dtype: int64

In [12]:
#filter data 

df = main_df[main_df['target'].isin(['Positive', 'Neutral', 'Negative'])]

In [13]:
len(df)

1587

## Check and Handle Empty Values

In [14]:
df.columns

Index(['communication', 'presentation', 'pace', 'examples', 'involvemen',
       'questions', 'objective', 'valuable information', 'sequence', 'apply',
       'e-learning', 'comment', 'target'],
      dtype='object')

In [15]:
col = ['communication','presentation', 'pace', 'examples','involvemen', 'questions','objective','valuable information','sequence','apply','e-learning']

df['null'] = df[col].isnull().sum(axis = 1)


In [16]:
len(df[df['null']>0])

15

In [17]:
len(df[df['null']>1])

4

### We can remove the rows which have more than 1 empty cell (4 Rows)
### And for the rows which have 1 empty cell, we can replace it with the most freuqent item (11 Rows)

In [18]:
df = df[df['null']<2]
len(df)

1583

In [19]:
df = df.fillna(df.mode().iloc[0])

In [20]:
df[df['null']>0].isnull().sum()

communication           0
presentation            0
pace                    0
examples                0
involvemen              0
questions               0
objective               0
valuable information    0
sequence                0
apply                   0
e-learning              0
comment                 0
target                  0
null                    0
dtype: int64

In [21]:
df.drop(['null'] ,axis = 1, inplace= True)

In [22]:
df['target'].value_counts()

Positive    1093
Neutral      461
Negative      29
Name: target, dtype: int64

## Need to add dummy data for "Negative" and "Neutral" to avoid imbalanced data

In [23]:
extra = pd.read_excel('Assignment_Extra_Data.xlsx')

In [24]:
extra.sample(10)

,comment,target
282,The instructor did not provide enough opportun...,Negative
256,The instructor did not provide enough opportun...,Negative
708,The training was not well-received by particip...,Negative
864,The training was too basic and could have cove...,Neutral
933,The training was too noisy.,Negative
51,It would be beneficial to have more opportunit...,Neutral
829,The training was not well-taught.,Negative
491,The training material was outdated.,Negative
582,The training was not well-coordinated with oth...,Negative
498,The training materials were poorly designed an...,Negative


In [25]:
# append the extra data with df

df = df.append(extra, ignore_index=True)

In [26]:
len(df)

2575

In [27]:
df['target'].value_counts()

Positive    1093
Negative     768
Neutral      714
Name: target, dtype: int64

# OrdinalEncoder for Target

In [28]:
enc = OrdinalEncoder()
df['target_enc']= (enc.fit_transform(df[['target']])).astype(int)

In [29]:
enc.categories_

[array(['Negative', 'Neutral', 'Positive'], dtype=object)]

# <font color='red'>Sentiment Analysis</font>

In [30]:
data = df[['comment', 'target_enc']]
data

,comment,target_enc
0,I learned a lot on this training,2
1,none,1
2,EXCELLENT COURSE,2
3,Good,1
4,Good Session,2
...,...,...
2570,Uipath traning,1
2571,Use of other media such as Videos,1
2572,we need more Training like this one,1
2573,Would be nice if there are physical training p...,1


In [31]:
nltk.download("stopwords")
stop = stopwords.words("English")
l_stem =LancasterStemmer()
p_stem = PorterStemmer()
lem = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soada\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
def preprocessing_l_stem(text):
    text = text.lower()
    pattern = "[a-zA-Z]+"
    token = re.findall(pattern, text)
    token = list(set(token))
    final_token =[]
    for i in token:
        if i not in stop:
            stem = l_stem.stem(i)
            final_token.append(stem)
    return final_token
    

In [33]:
def preprocessing_p_stem(text):
    text = text.lower()
    pattern = "[a-zA-Z]+"
    token = re.findall(pattern, text)
    token = list(set(token))
    final_token =[]
    for i in token:
        if i not in stop:
            stem = p_stem.stem(i)
            final_token.append(stem)
    return final_token
    

In [34]:
def preprocessing_lem(text):
    text = text.lower()
    pattern = "[a-zA-Z]+"
    token = re.findall(pattern, text)
    token = list(set(token))
    final_token =[]
    for i in token:
        if i not in stop:
            lemm = lem.lemmatize(i)
            final_token.append(lemm)
    return final_token

In [35]:
# error due to data type

data['comment'] = data['comment'].astype(str)

In [36]:
data['l_stem'] = data['comment'].apply(preprocessing_l_stem)
data['p_stem'] = data['comment'].apply(preprocessing_p_stem)
data['lem'] = data['comment'].apply(preprocessing_lem)

data

,comment,target_enc,l_stem,p_stem,lem
0,I learned a lot on this training,2,"[learn, lot, train]","[learn, lot, train]","[learned, lot, training]"
1,none,1,[non],[none],[none]
2,EXCELLENT COURSE,2,"[excel, cours]","[excel, cours]","[excellent, course]"
3,Good,1,[good],[good],[good]
4,Good Session,2,"[good, sess]","[good, session]","[good, session]"
...,...,...,...,...,...
2570,Uipath traning,1,"[uipa, tran]","[uipath, trane]","[uipath, traning]"
2571,Use of other media such as Videos,1,"[video, med, us]","[video, media, use]","[video, medium, use]"
2572,we need more Training like this one,1,"[nee, lik, train, on]","[need, like, train, one]","[need, like, training, one]"
2573,Would be nice if there are physical training p...,1,"[focus, would, platform, nic, phys, train, sess]","[focus, would, platform, nice, physic, train, ...","[focused, would, platform, nice, physical, tra..."


# TF-IDF

In [37]:
tfidf = TfidfVectorizer()

# <font color='orange'> 1- LancasterStemmer Data</font>

In [38]:
df_Lstem = data[['l_stem', 'target_enc']]
df_Lstem

,l_stem,target_enc
0,"[learn, lot, train]",2
1,[non],1
2,"[excel, cours]",2
3,[good],1
4,"[good, sess]",2
...,...,...
2570,"[uipa, tran]",1
2571,"[video, med, us]",1
2572,"[nee, lik, train, on]",1
2573,"[focus, would, platform, nic, phys, train, sess]",1


In [39]:
text_lstem = [" ".join(i) for i in list(df_Lstem['l_stem'])]

In [40]:
len(text_lstem)

2575

In [41]:
tfidf_text_lstem = tfidf.fit_transform(text_lstem)

In [42]:
tfidf.get_feature_names_out()

array(['ab', 'abba', 'abl', 'absorb', 'academy', 'acc', 'access',
       'accommod', 'across', 'act', 'ad', 'adapt', 'address', 'adequ',
       'adjust', 'admin', 'adv', 'advert', 'afford', 'ahm', 'ai', 'al',
       'ala', 'alaa', 'align', 'alloc', 'allow', 'almost', 'alobeidl',
       'along', 'alot', 'already', 'also', 'alway', 'amaz', 'ampl',
       'analys', 'angry', 'anoth', 'answ', 'anyth', 'anytim', 'appl',
       'apply', 'apprecy', 'approach', 'approv', 'are', 'area', 'arrang',
       'ask', 'aspir', 'assess', 'assign', 'at', 'attend', 'attitud',
       'audy', 'avail', 'avoid', 'aw', 'awesom', 'background', 'bad',
       'bal', 'bas', 'bay', 'becom', 'begin', 'believ', 'benef',
       'benefit', 'best', 'bet', 'beyond', 'bi', 'bias', 'bit', 'blend',
       'bless', 'block', 'book', 'brand', 'break', 'build', 'busy',
       'cabl', 'cam', 'car', 'cas', 'catch', 'caus', 'cert', 'chain',
       'challeng', 'chandr', 'chang', 'chaot', 'chatbot', 'chatgpt',
       'check', 'chos',

In [43]:
tfidf_text_lstem.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
df_tfidf_lstem = pd.DataFrame(tfidf_text_lstem.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf_lstem

,ab,abba,abl,absorb,academy,acc,access,accommod,across,act,...,wor,work,workshop,world,worthy,would,writ,ye,yor,yr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.298995,0.0,0.0,0.0,0.0


## Splitting Data

In [45]:
xtrain, xtest, ytrain, ytest = train_test_split(df_tfidf_lstem,df_Lstem['target_enc'], random_state=1,
                                                test_size=0.1)


In [46]:
print(f"x_train shape is > {xtrain.shape}")
print(f"y_train shape is > {ytrain.shape}")
print(f"x_test shape is > {xtest.shape}")
print(f"y_test shape is > {ytest.shape}")

x_train shape is > (2317, 820)
y_train shape is > (2317,)
x_test shape is > (258, 820)
y_test shape is > (258,)


## Model:

##  1 - Naiev Bais

In [47]:
nb = MultinomialNB()
nb.fit(xtrain, ytrain)
nb_ypred = nb.predict(xtest)

In [48]:
accuracy_score(ytest,nb_ypred)

0.8410852713178295

In [49]:
confusion_matrix(ytest, nb_ypred)

array([[65,  1,  3],
       [ 2, 53, 31],
       [ 0,  4, 99]], dtype=int64)

In [50]:
print(classification_report(ytest,nb_ypred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96        69
           1       0.91      0.62      0.74        86
           2       0.74      0.96      0.84       103

    accuracy                           0.84       258
   macro avg       0.88      0.84      0.84       258
weighted avg       0.86      0.84      0.84       258



## 2 - Logistic regression

In [51]:
lr = LogisticRegression()
lr.fit(xtrain, ytrain)
lr_ypred = lr.predict(xtest)

In [52]:
accuracy_score(ytest,lr_ypred)

0.9186046511627907

In [53]:
confusion_matrix(ytest,lr_ypred)

array([[65,  3,  1],
       [ 1, 78,  7],
       [ 1,  8, 94]], dtype=int64)

In [54]:
print(classification_report(ytest,lr_ypred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96        69
           1       0.88      0.91      0.89        86
           2       0.92      0.91      0.92       103

    accuracy                           0.92       258
   macro avg       0.92      0.92      0.92       258
weighted avg       0.92      0.92      0.92       258



## 3 - Decision Tree

In [55]:
dt = DecisionTreeClassifier()
dt.fit(xtrain, ytrain)
dt_ypred = lr.predict(xtest)

In [56]:
accuracy_score(ytest,dt_ypred)

0.9186046511627907

In [57]:
confusion_matrix(ytest,dt_ypred)

array([[65,  3,  1],
       [ 1, 78,  7],
       [ 1,  8, 94]], dtype=int64)

In [58]:
print(classification_report(ytest,dt_ypred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96        69
           1       0.88      0.91      0.89        86
           2       0.92      0.91      0.92       103

    accuracy                           0.92       258
   macro avg       0.92      0.92      0.92       258
weighted avg       0.92      0.92      0.92       258



## 4- KNN

In [59]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(xtrain, ytrain)
knn_ypred = knn.predict(xtest)

In [60]:
accuracy_score(ytest,knn_ypred)

0.7868217054263565

In [61]:
confusion_matrix(ytest,knn_ypred)

array([[45, 23,  1],
       [ 1, 79,  6],
       [ 0, 24, 79]], dtype=int64)

In [62]:
print(classification_report(ytest, knn_ypred))

              precision    recall  f1-score   support

           0       0.98      0.65      0.78        69
           1       0.63      0.92      0.75        86
           2       0.92      0.77      0.84       103

    accuracy                           0.79       258
   macro avg       0.84      0.78      0.79       258
weighted avg       0.84      0.79      0.79       258



# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------

# <font color='orange'> 2- PorterStemmer Data</font>

In [63]:
df_pstem = data[['p_stem', 'target_enc']]
df_pstem

,p_stem,target_enc
0,"[learn, lot, train]",2
1,[none],1
2,"[excel, cours]",2
3,[good],1
4,"[good, session]",2
...,...,...
2570,"[uipath, trane]",1
2571,"[video, media, use]",1
2572,"[need, like, train, one]",1
2573,"[focus, would, platform, nice, physic, train, ...",1


In [64]:
text_pstem = [" ".join(i) for i in list(df_pstem['p_stem'])]

In [65]:
len(text_pstem)

2575

In [66]:
tfidf_text_pstem = tfidf.fit_transform(text_pstem)

In [67]:
tfidf.get_feature_names_out()

array(['abba', 'abl', 'absorb', 'abul', 'academi', 'access', 'accommod',
       'accur', 'across', 'activ', 'actual', 'ad', 'adapt', 'add',
       'address', 'adequ', 'adjust', 'administ', 'advanc', 'advertis',
       'advic', 'advis', 'afford', 'ahm', 'ai', 'al', 'alaa', 'alaaal',
       'ali', 'align', 'alloc', 'allow', 'almost', 'alobeidli', 'along',
       'alot', 'alreadi', 'also', 'alway', 'amaz', 'ampl', 'analysi',
       'angri', 'anoth', 'answer', 'anyth', 'anytim', 'appli', 'applic',
       'appreci', 'approach', 'approv', 'area', 'arrang', 'ask', 'aspir',
       'assess', 'assign', 'attend', 'attent', 'attitud', 'audienc',
       'avail', 'avoid', 'aw', 'awar', 'awesom', 'background', 'bad',
       'balanc', 'base', 'basi', 'basic', 'bayan', 'becom', 'beginn',
       'believ', 'benefici', 'benefit', 'best', 'better', 'beyond', 'bi',
       'bias', 'bit', 'blend', 'bless', 'block', 'book', 'brand', 'break',
       'build', 'busi', 'cabl', 'came', 'career', 'carer', 'case',
  

In [68]:
tfidf_text_pstem.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
df_tfidf_pstem = pd.DataFrame(tfidf_text_pstem.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf_pstem

,abba,abl,absorb,abul,academi,access,accommod,accur,across,activ,...,work,workshop,world,worth,worthi,would,write,ye,yor,yr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.298453,0.0,0.0,0.0,0.0


## Splitting Data

In [70]:
xtrain, xtest, ytrain, ytest = train_test_split(df_tfidf_pstem,df_Lstem['target_enc'], random_state=1,
                                                test_size=0.1)


In [71]:
print(f"x_train shape is > {xtrain.shape}")
print(f"y_train shape is > {ytrain.shape}")
print(f"x_test shape is > {xtest.shape}")
print(f"y_test shape is > {ytest.shape}")

x_train shape is > (2317, 890)
y_train shape is > (2317,)
x_test shape is > (258, 890)
y_test shape is > (258,)


## Model:

##  1 - Naiev Bais

In [72]:
nb = MultinomialNB()
nb.fit(xtrain, ytrain)
nb_ypred = nb.predict(xtest)

In [73]:
accuracy_score(ytest,nb_ypred)

0.8372093023255814

In [74]:
confusion_matrix(ytest, nb_ypred)

array([[64,  1,  4],
       [ 2, 53, 31],
       [ 0,  4, 99]], dtype=int64)

In [75]:
print(classification_report(ytest,nb_ypred))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95        69
           1       0.91      0.62      0.74        86
           2       0.74      0.96      0.84       103

    accuracy                           0.84       258
   macro avg       0.87      0.83      0.84       258
weighted avg       0.86      0.84      0.83       258



## 2 - Logistic regression

In [76]:
lr = LogisticRegression()
lr.fit(xtrain, ytrain)
lr_ypred = lr.predict(xtest)

In [77]:
accuracy_score(ytest,lr_ypred)

0.9147286821705426

In [78]:
confusion_matrix(ytest,lr_ypred)

array([[65,  3,  1],
       [ 2, 77,  7],
       [ 1,  8, 94]], dtype=int64)

In [79]:
print(classification_report(ytest,lr_ypred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        69
           1       0.88      0.90      0.89        86
           2       0.92      0.91      0.92       103

    accuracy                           0.91       258
   macro avg       0.92      0.92      0.92       258
weighted avg       0.92      0.91      0.91       258



## 3 - Decision Tree

In [80]:
dt = DecisionTreeClassifier()
dt.fit(xtrain, ytrain)
dt_ypred = lr.predict(xtest)

In [81]:
accuracy_score(ytest,dt_ypred)

0.9147286821705426

In [82]:
confusion_matrix(ytest,dt_ypred)

array([[65,  3,  1],
       [ 2, 77,  7],
       [ 1,  8, 94]], dtype=int64)

In [83]:
print(classification_report(ytest,dt_ypred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        69
           1       0.88      0.90      0.89        86
           2       0.92      0.91      0.92       103

    accuracy                           0.91       258
   macro avg       0.92      0.92      0.92       258
weighted avg       0.92      0.91      0.91       258



## 4- KNN

In [84]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(xtrain, ytrain)
knn_ypred = knn.predict(xtest)

In [85]:
accuracy_score(ytest,knn_ypred)

0.7945736434108527

In [86]:
confusion_matrix(ytest,knn_ypred)

array([[48, 20,  1],
       [ 1, 79,  6],
       [ 0, 25, 78]], dtype=int64)

In [87]:
print(classification_report(ytest, knn_ypred))

              precision    recall  f1-score   support

           0       0.98      0.70      0.81        69
           1       0.64      0.92      0.75        86
           2       0.92      0.76      0.83       103

    accuracy                           0.79       258
   macro avg       0.84      0.79      0.80       258
weighted avg       0.84      0.79      0.80       258



# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------

# <font color='orange'> 3- WordNetLemmatizer Data</font>

In [88]:
df_lem = data[['lem', 'target_enc']]
df_lem

,lem,target_enc
0,"[learned, lot, training]",2
1,[none],1
2,"[excellent, course]",2
3,[good],1
4,"[good, session]",2
...,...,...
2570,"[uipath, traning]",1
2571,"[video, medium, use]",1
2572,"[need, like, training, one]",1
2573,"[focused, would, platform, nice, physical, tra...",1


In [89]:
text_lem = [" ".join(i) for i in list(df_lem['lem'])]

In [90]:
len(text_lem)

2575

In [91]:
tfidf_text_lem = tfidf.fit_transform(text_lem)

In [92]:
tfidf.get_feature_names_out()

array(['abbas', 'able', 'absorb', ..., 'yes', 'yor', 'yr'], dtype=object)

In [93]:
tfidf_text_lem.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [94]:
df_tfidf_lem = pd.DataFrame(tfidf_text_lem.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf_lem

,abbas,able,absorb,abul,academy,access,accommodate,accommodating,accuring,across,...,world,worth,worthy,would,write,writing,ye,yes,yor,yr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.297068,0.0,0.0,0.0,0.0,0.0,0.0


## Splitting Data

In [95]:
xtrain, xtest, ytrain, ytest = train_test_split(tfidf_text_lem,df_lem['target_enc'], random_state=1,
                                                test_size=0.1)


In [96]:
print(f"x_train shape is > {xtrain.shape}")
print(f"y_train shape is > {ytrain.shape}")
print(f"x_test shape is > {xtest.shape}")
print(f"y_test shape is > {ytest.shape}")

x_train shape is > (2317, 1094)
y_train shape is > (2317,)
x_test shape is > (258, 1094)
y_test shape is > (258,)


## Model:

##  1 - Naiev Bais

In [97]:
nb = MultinomialNB()
nb.fit(xtrain, ytrain)
nb_ypred = nb.predict(xtest)

In [98]:
accuracy_score(ytest,nb_ypred)

0.8372093023255814

In [99]:
confusion_matrix(ytest, nb_ypred)

array([[65,  1,  3],
       [ 2, 52, 32],
       [ 1,  3, 99]], dtype=int64)

In [100]:
print(classification_report(ytest,nb_ypred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        69
           1       0.93      0.60      0.73        86
           2       0.74      0.96      0.84       103

    accuracy                           0.84       258
   macro avg       0.87      0.84      0.84       258
weighted avg       0.86      0.84      0.83       258



## 2 - Logistic regression

In [101]:
lr = LogisticRegression()
lr.fit(xtrain, ytrain)
lr_ypred = lr.predict(xtest)

In [102]:
accuracy_score(ytest,lr_ypred)

0.9031007751937985

In [103]:
confusion_matrix(ytest,lr_ypred)

array([[64,  4,  1],
       [ 2, 76,  8],
       [ 1,  9, 93]], dtype=int64)

In [104]:
print(classification_report(ytest,lr_ypred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94        69
           1       0.85      0.88      0.87        86
           2       0.91      0.90      0.91       103

    accuracy                           0.90       258
   macro avg       0.91      0.90      0.91       258
weighted avg       0.90      0.90      0.90       258



## 3 - Decision Tree

In [105]:
dt = DecisionTreeClassifier()
dt.fit(xtrain, ytrain)
dt_ypred = lr.predict(xtest)

In [106]:
accuracy_score(ytest,dt_ypred)

0.9031007751937985

In [107]:
confusion_matrix(ytest,dt_ypred)

array([[64,  4,  1],
       [ 2, 76,  8],
       [ 1,  9, 93]], dtype=int64)

In [108]:
print(classification_report(ytest,dt_ypred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94        69
           1       0.85      0.88      0.87        86
           2       0.91      0.90      0.91       103

    accuracy                           0.90       258
   macro avg       0.91      0.90      0.91       258
weighted avg       0.90      0.90      0.90       258



## 4- KNN

In [109]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(xtrain, ytrain)
knn_ypred = knn.predict(xtest)

In [110]:
accuracy_score(ytest,knn_ypred)

0.7790697674418605

In [111]:
confusion_matrix(ytest,knn_ypred)

array([[44, 25,  0],
       [ 1, 79,  6],
       [ 0, 25, 78]], dtype=int64)

In [112]:
print(classification_report(ytest, knn_ypred))

              precision    recall  f1-score   support

           0       0.98      0.64      0.77        69
           1       0.61      0.92      0.73        86
           2       0.93      0.76      0.83       103

    accuracy                           0.78       258
   macro avg       0.84      0.77      0.78       258
weighted avg       0.84      0.78      0.78       258



# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------

# Conclusion 

## 1- The data is very imbalanced specially for "Negative" comment target,
## so we added extra data with "Negative" and "Neutral" tagging 
##
## 2- The best method of data stemming and lemmatization is LancasterStemmer
##
## 3- The best result is shared between Logistic Regression and Decision Tree
## Both provid 92% accuracy, and 92% f1 score for "Postive" target,
## 89% f1 score for "Neutral" target and 96% f1 score for "Negative" target
##
## 4- So we will use those two models to predict the score for comments with no target, 
## and will validate sample of it manually to check its accuracy

# ------------------------------------------------------------------------------

In [113]:
df_null = pd.DataFrame(main_df[['comment', 'target']].copy())

In [114]:
len(df_null)

2394

In [115]:
df_null.head()

,comment,target
0,I learned a lot on this training,Positive
1,none,Neutral
2,EXCELLENT COURSE,Positive
3,NaN,Neutral
4,Good Session,Positive


In [116]:
# Removed target data

values_to_remove = ['Positive', 'Negative', 'Neutral']
df_null = df_null[~df_null.target.isin(values_to_remove)]
df_null

,comment,target
9,Maybe a recording will allow the participants ...,NaN
21,NaN,NaN
22,Excellent training on new technology and learn...,NaN
26,"it's very long, and the last part is the most ...",NaN
28,Thanks a lot for the great and interesting tra...,NaN
...,...,...
2389,It was Great opportunity to understand more ab...,NaN
2390,NaN,NaN
2391,NaN,NaN
2392,NaN,NaN


In [117]:
len(df_null)

807

In [118]:
df_null = df_null[['comment']].astype(str)

In [119]:
df_null['len'] = [len(i) for i in df_null["comment"]]

In [120]:
df_null

,comment,len
9,Maybe a recording will allow the participants ...,89
21,nan,3
22,Excellent training on new technology and learn...,63
26,"it's very long, and the last part is the most ...",115
28,Thanks a lot for the great and interesting tra...,85
...,...,...
2389,It was Great opportunity to understand more ab...,88
2390,nan,3
2391,nan,3
2392,nan,3


In [121]:
temp = df_null[df_null['len'] <5]
temp['comment'].unique()

array(['nan', '.', 'G', 'N/L', 'Nill', 'Ok', 'Na', 'V', 'na', 'NL', 'nil',
       'Null', 'Nil', 'Nli', 'Sure', 'Yes', 'Good', 'good', 'Done',
       'Nice', 'N0', 'htt', 'W'], dtype=object)

In [122]:
temp = df_null[df_null['len'] <4]
temp['comment'].unique()

array(['nan', '.', 'G', 'N/L', 'Ok', 'Na', 'V', 'na', 'NL', 'nil', 'Nil',
       'Nli', 'Yes', 'N0', 'htt', 'W'], dtype=object)

### Will remove the rows less than 4, and remove "Nill" & "Null" manually

In [123]:
len(df_null)

807

In [124]:
df_null = df_null[df_null['len'] >3]
len(df_null)

436

In [125]:
comment_to_remove = ['Null', 'Nill']

df_null = df_null[~df_null.comment.isin(comment_to_remove)]
len(df_null)

433

In [126]:
df_null[df_null['len']<6]

,comment,len
753,Sure,4
1219,Good,4
1223,good,4
1234,Done,4
1245,Good,4
1264,Good,4
1273,good,4
1278,Good,4
1299,Good,4
1305,good,4


### Preprocess the data

In [127]:
df_null['l_stem'] = df_null['comment'].apply(preprocessing_l_stem)

df_null

,comment,len,l_stem
9,Maybe a recording will allow the participants ...,89,"[particul, mayb, particip, top, allow, back, r..."
22,Excellent training on new technology and learn...,63,"[many, excel, learnt, new, insight, train, tec..."
26,"it's very long, and the last part is the most ...",115,"[quick, tim, import, part, pref, last, long, mak]"
28,Thanks a lot for the great and interesting tra...,85,"[gre, lot, interest, thank, apprecy, much, tra..."
29,Excellent session. Maybe spread over two days ...,94,"[mayb, pract, excel, see, spread, sess, two, t..."
...,...,...,...
2309,Great knowledge gaind today.,28,"[gre, gaind, knowledg, today]"
2336,The session was very helpful I will appreciate...,77,"[acn, help, ev, sess, apprecy, held, tim]"
2387,Great Experience in short time,30,"[gre, short, tim, expery]"
2388,Tnank You,9,[tnank]


In [128]:
text_null_l_stem = [" ".join(i) for i in list(df_null['l_stem'])]

In [129]:
text = tfidf.fit_transform(text_null_l_stem)

In [130]:
text.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [131]:
df_tfidf_null_l_stem = pd.DataFrame(text.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf_null_l_stem

,ab,abba,abl,access,accommod,acn,acquir,across,act,ad,...,whol,wirh,wish,without,work,workshop,would,writ,ye,yr
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,0.0,0.0,0.0,0.0,0.0,0.519592,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
430,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
431,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Faced issue with total feature required while running Logistic regression, so I created dummy data

In [132]:
col = []

for i in range(1,497):
    i = str(i)
    col_num = "ab"+i
    col.append(col_num)
    

In [133]:
d = pd.DataFrame(0, index=np.arange(len(df_null)), columns=col)
d.head()

,ab1,ab2,ab3,ab4,ab5,ab6,ab7,ab8,ab9,ab10,...,ab487,ab488,ab489,ab490,ab491,ab492,ab493,ab494,ab495,ab496
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
df_tfidf_null_l_stem = result = pd.concat([df_tfidf_null_l_stem, d], axis=1)
df_tfidf_null_l_stem

,ab,abba,abl,access,accommod,acn,acquir,across,act,ad,...,ab487,ab488,ab489,ab490,ab491,ab492,ab493,ab494,ab495,ab496
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
429,0.0,0.0,0.0,0.0,0.0,0.519592,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
430,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
431,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


## Logistic Regression

In [135]:
y_pred_lr = lr.predict(df_tfidf_null_l_stem)

In [136]:
len(y_pred_lr)

433

In [137]:
y_pred_lr

array([1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [138]:
df_null['lr_predect'] = y_pred_lr

In [139]:
df_null

,comment,len,l_stem,lr_predect
9,Maybe a recording will allow the participants ...,89,"[particul, mayb, particip, top, allow, back, r...",1
22,Excellent training on new technology and learn...,63,"[many, excel, learnt, new, insight, train, tec...",2
26,"it's very long, and the last part is the most ...",115,"[quick, tim, import, part, pref, last, long, mak]",2
28,Thanks a lot for the great and interesting tra...,85,"[gre, lot, interest, thank, apprecy, much, tra...",1
29,Excellent session. Maybe spread over two days ...,94,"[mayb, pract, excel, see, spread, sess, two, t...",1
...,...,...,...,...
2309,Great knowledge gaind today.,28,"[gre, gaind, knowledg, today]",1
2336,The session was very helpful I will appreciate...,77,"[acn, help, ev, sess, apprecy, held, tim]",1
2387,Great Experience in short time,30,"[gre, short, tim, expery]",1
2388,Tnank You,9,[tnank],1


## Decision Tree 

In [140]:
y_pred_dt = dt.predict(df_tfidf_null_l_stem)

In [141]:
len(y_pred_dt)

433

In [142]:
y_pred_dt

array([2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 2, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1,
       2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,

In [143]:
df_null['dt_predect'] = y_pred_dt

## Checking sample manually

In [144]:
df_null.drop(['len', 'l_stem'], inplace=True, axis = 1)

In [145]:
df_null['variance'] = df_null['lr_predect'] - df_null['dt_predect']
df_null

,comment,lr_predect,dt_predect,variance
9,Maybe a recording will allow the participants ...,1,2,-1
22,Excellent training on new technology and learn...,2,1,1
26,"it's very long, and the last part is the most ...",2,2,0
28,Thanks a lot for the great and interesting tra...,1,1,0
29,Excellent session. Maybe spread over two days ...,1,1,0
...,...,...,...,...
2309,Great knowledge gaind today.,1,1,0
2336,The session was very helpful I will appreciate...,1,1,0
2387,Great Experience in short time,1,1,0
2388,Tnank You,1,1,0


In [146]:
variance = df_null[df_null['variance']!= 0]
len(variance)

55

# Both models predicted the same score for 87% of the data, let's check
# sample of 55 cases manually

In [147]:
variance = variance.reset_index()

In [148]:
pd.set_option('display.max_colwidth', None)

variance[:10]


,index,comment,lr_predect,dt_predect,variance
0,9,Maybe a recording will allow the participants to go back to a particular topic and review,1,2,-1
1,22,Excellent training on new technology and learnt many insights..,2,1,1
2,79,The topic was great. Instructor is very passionate about the topic.,1,2,-1
3,80,Please divide the session in two and make this session a learning not a walk thru. it was good overall but the content was so much that trainner had no option except to run thru the content.,2,1,1
4,87,"Software installation should have been informed atleast 3 days prior as IT Team software installation takes time ., otherwise the Training are good",1,2,-1
5,95,Very Insightful training and a knowledgeable trainer. Maybe during the later cxourse of the session I feel the trainer was rushed to complete the content but overall it was really really good.,2,1,1
6,117,We do not have access to install the software system on corporate PC,2,1,1
7,319,Interesting training. Would like to know more â¤ï¸,1,0,1
8,383,"Well co-ordination and interaction with everyone , encouraging each one to know about the concepts and techniques",1,2,-1
9,404,Very informative session which are discussed need for my work. But if we have 3 training sessions for this ipv4 we can improve more.,1,2,-1


In [149]:
variance[30:40]

,index,comment,lr_predect,dt_predect,variance
30,1218,Nice presentation,2,1,1
31,1280,Session very useful to understand Arcadiyan Next Generation Gateway,1,2,-1
32,1282,Need to given the training at same time all technician,2,1,1
33,1301,it was good session on the subject topic . Keep it up!!,1,2,-1
34,1355,Mashallah nice,1,2,-1
35,1524,Please to see the possibility of reducing session duration less than 6 hours / that course may need one more day for more practice / what about schedule training on weekeends (one day only out of 3 days),1,2,-1
36,1569,The training was helpfull and we enjoyed. \nMr Eslam cover and give us his time to transfer the topics.\nvery happy with,1,2,-1
37,1733,We got professional training by trainer and its easy to understand and grasp the concepts efficiently.Now i am having good knowledge and confident to do cloud projects .Except training he is a good motivator to study technology and enhance our skills.It was a great pleasure to listen to your talk - highly inspirational and perfectly pitched to the audience as always. Looking forward more training from him.,2,1,1
38,1927,Well explained the topics,1,2,-1
39,1932,Training should be given more time,2,1,1


In [150]:
variance[41:]

,index,comment,lr_predect,dt_predect,variance
41,1962,GOOD TOPIC AND NICE PRESENTATIONS,2,1,1
42,2046,it was good experience to learn basic and now I am planning to joint more training related to python and machine learning.,1,2,-1
43,2048,Training explained all the basic requirement for pythons .thanks,1,2,-1
44,2049,I like a enroll for other courses,1,0,1
45,2089,"Training was very helpful, just want to add should be held of off days",2,1,1
46,2102,I feel good training and very useful,2,1,1
47,2121,Excellent training. Required lab for better understanding,1,2,-1
48,2125,Great Introduction to AI,2,1,1
49,2132,NOW GOOD TO ANALISE WHETHER IT IS AI OR NOT ATLEAST WHEN I SEE SOME APP ETC...,1,2,-1
50,2134,Lab was very good.,1,2,-1


## Both Models are good :)

# ---------------------------------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------------------------------

# Thank You!

#
#
#